<a href="https://colab.research.google.com/github/excellentwork/data-sourcing-challenge/blob/main/retrieve_movie_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
params = {
  'api-key': nyt_api_key,
  'fq': filter_query,
  'sort': sort,
  'fl': field_list,
  'begin_date': begin_date,
  'end_date': end_date,
  }


In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_params = {
        'api-key': userdata.get('NYT_API_KEY'),
        'fq': filter_query,
        'sort': sort,
        'fl': field_list,
        'begin_date': begin_date,
        'end_date': end_date,
        'page': page
    }

    # Make a "GET" request and retrieve the JSON
    response = requests.get(url, params=query_params)
    reviews = response.json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)


    #Try and save the reviews to the reviews_list
    try:
      if reviews["response"]["docs"]:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
          reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Checked page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No results on page {page}, stopping.")
        break

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
reviews_list_strings = [json.dumps(movie, indent=4) for movie in reviews_list[:5]]

# Print the results to see the JSON strings of the first 5 entries
for movie_str in reviews_list_strings:
    print(movie_str)


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

# Print the DataFrame
reviews_df

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# End string should include " Review" to avoid cutting title early

# Create a new column 'title' by extracting the title using a lambda function
reviews_df['title'] = reviews_df['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.rfind("\u2019 Review")]
)



# Print the DataFrame with the new column
reviews_df

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};"
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    headers = {
      "accept": "application/json",
      "Authorization": f"Bearer {tmdb_access_token}"
    }

    response_general_query = requests.get(
      f"https://api.themoviedb.org/3/search/movie?query={title}",
      headers=headers
    )
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

    try:
        # Get movie id
        response_movies_id = response_general_query.json()["results"][0]["id"]

        # Make a request for a the full movie details
        # Execute "GET" request with url
        response_full_details = requests.get(
          f"https://api.themoviedb.org/3/movie/{response_movies_id}",
          headers=headers
        )
        # Extract the genre names into a list
        genres = extract_genres(response_full_details.json()["genres"])

        # Extract the spoken_languages' English name into a list
        spoken_languages = extract_english_name(response_full_details.json()["spoken_languages"])

        # Extract the production_countries' name into a list
        production_countries = extract_production_country(response_full_details.json()["production_countries"])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": response_full_details.json()['title'],
            "original_title": response_full_details.json()['original_title'],
            "budget": response_full_details.json()["budget"],
            "original_language": response_full_details.json()["original_language"],
            "homepage":response_full_details.json()["homepage"],
            "overview": response_full_details.json()["overview"],
            "popularity": response_full_details.json()["popularity"],
            "runtime" : response_full_details.json()["runtime"],
            "revenue": response_full_details.json()["revenue"],
            "release_date": response_full_details.json()["release_date"],
            "vote_average": response_full_details.json()["vote_average"],
            "vote_count": response_full_details.json()["vote_count"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries

        })

        # Print out the title that was found
        print(f"Found {title}")
    except IndexError:
        print(f"{title} not found.")

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
tmdb_movies_list_strings = [json.dumps(movie, indent=4) for movie in tmdb_movies_list[:5]]

# Print the results to see the JSON strings
for movie_str in tmdb_movies_list_strings:
    print(movie_str)

In [ ]:
# Convert the results to a DataFrame
tmdb_search_results_df = pd.json_normalize(tmdb_movies_list)
tmdb_search_results_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_search_results_df, on="title", how="inner")
merged_df

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["spoken_languages", "genres", "production_countries"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str).replace(r"[\[\]'']", "", regex=True)


# Display the fixed DataFrame
merged_df

In [ ]:
# Drop "byline.person" column
merged_df.drop('byline.person', axis=1, inplace=True)
merged_df

In [ ]:
# Delete duplicate rows and reset index

# Convert the 'web_url' column to a hashable type, such as string
print(type(merged_df['web_url']))
merged_df['web_url'] = merged_df['web_url'].astype(str)

# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates(subset='web_url', keep='first').reset_index(drop=True)
merged_df

In [ ]:
# Export data to CSV without the index
os.makedirs('output')
merged_df.to_csv('output/collected_data.csv', index=False)